In [1]:
require 'hdf5';
require 'rnn';

In [2]:
myFile = hdf5.open('../data_preprocessed/6-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [3]:
data['input_data_train']:size(1)-4

599905	


In [11]:
train = data['input_data_train']:narrow(1,5,data['input_data_train']:size(1)-1509)
out = data['output_matrix_train']:narrow(1,1,data['input_data_train']:size(1)-1509)

In [19]:
nchar = 49
l = 50
b = 16
n = train:size(1)
train_matrix = train:view(b,n/b):type('torch.DoubleTensor')
out_matrix = out:view(b,n/b):type('torch.DoubleTensor')

In [23]:
embed_dim = 20
LT = nn.LookupTable(nchar, embed_dim)

r = nn.Recurrent(
   embed_dim, nn.Linear(embed_dim, embed_dim), 
   nn.Linear(embed_dim, embed_dim), nn.Tanh(),l
)

rnn = nn.Sequential()
   :add(LT) 
   :add(nn.SplitTable(1,10))
   :add(nn.Sequencer(r))
   :add(nn.Sequencer(nn.Linear(embed_dim,2)))
   :add(nn.Sequencer(nn.LogSoftMax()))

rnn:remember('both')

crit = nn.SequencerCriterion(nn.ClassNLLCriterion())

params, grad_params = rnn:getParameters()

lr = 0.5

In [24]:
for i = 1, params:size(1) do
    params[i] = torch.uniform(-0.05,0.05)
end

In [25]:
-- To store the loss
av_L = 0
old_L = 0
new_L = 0
c = 0.1
for i = 1, 20 do
    -- timing the epoch
    timer = torch.Timer()
    
    old_L = new_L
    
    av_L = 0

    -- mini batch loop
    for t = 1, n/(b*l) do
        -- Mini batch data
        inputs_batch = train_matrix:narrow(2,(t-1)*l+1,l)
        targets_batch = out_matrix:narrow(2,(t-1)*l+1,l)
        
        tab = torch.split(targets_batch,1,2)
        target_table = {}
        for k, v in pairs(tab) do
          target_table[k] = v:squeeze()
        end 
        
        -- reset gradients
        grad_params:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs = rnn:forward(inputs_batch:t()) 

        -- Average loss computation
        f = crit:forward(outputs, target_table)
        av_L = av_L +f

        -- Backward pass
        df_do = crit:backward(outputs,target_table)
        rnn:backward(inputs_batch, df_do)
        
        grad_params:view(grad_params:size(1),1):renorm(1,2,5)
        
        rnn:updateParameters(lr)

    end
    
    new_L = av_L/math.floor(train:size(1)/b)
    
    print('Epoch '..i..': '..timer:time().real)
    print('Average Loss: '..new_L)
    
    if (new_L-old_L)< c then 
        lr = lr/2 
        c = c/2
    end
    
    if c < 0.001 then c = 0.1 end
    
end



Epoch 1: 16.774312973022	
Average Loss: 0.31411771687415	


Epoch 2: 16.888953208923	
Average Loss: 0.25923890417995	


Epoch 3: 13.90478515625	
Average Loss: 0.23034241823792	


Epoch 4: 14.78458404541	
Average Loss: 0.21935126587833	


Epoch 5: 15.43119096756	
Average Loss: 0.21465063898723	


...

Average Loss: 0.21048265858625	


Epoch 17: 15.517771959305	
Average Loss: 0.21048152195259	


Epoch 18: 16.392879009247	
Average Loss: 0.21048095342564	


Epoch 19: 14.436084985733	
Average Loss: 0.21048066910843	


Epoch 20: 14.600162982941	
Average Loss: 0.21048052693625	


### LSTM:

In [26]:
embed_dim = 20
LT_lstm = nn.LookupTable(nchar, embed_dim)

r = nn.Recurrent(
   embed_dim, nn.Linear(embed_dim, embed_dim), 
   nn.Linear(embed_dim, embed_dim), nn.Tanh(),l
)

rnn_lstm = nn.Sequential()
   :add(LT) 
   :add(nn.SplitTable(1,10))
   :add(nn.Sequencer(nn.LSTM(embed_dim,embed_embed_dim)))
   :add(nn.Sequencer(nn.Linear(embed_dim,2)))
   :add(nn.Sequencer(nn.LogSoftMax()))

rnn_lstm:remember('both')

crit = nn.SequencerCriterion(nn.ClassNLLCriterion())

params, grad_params = rnn_lstm:getParameters()

lr = 0.5

In [27]:
for i = 1, params:size(1) do
    params[i] = torch.uniform(-0.05,0.05)
end

In [28]:
-- To store the loss
av_L = 0
old_L = 0
new_L = 0
c = 0.1
for i = 1, 20 do
    -- timing the epoch
    timer = torch.Timer()
    
    old_L = new_L
    
    av_L = 0

    -- mini batch loop
    for t = 1, n/(b*l) do
        -- Mini batch data
        inputs_batch = train_matrix:narrow(2,(t-1)*l+1,l)
        targets_batch = out_matrix:narrow(2,(t-1)*l+1,l)
        
        tab = torch.split(targets_batch,1,2)
        target_table = {}
        for k, v in pairs(tab) do
          target_table[k] = v:squeeze()
        end 
        
        -- reset gradients
        grad_params:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs = rnn_lstm:forward(inputs_batch:t()) 

        -- Average loss computation
        f = crit:forward(outputs, target_table)
        av_L = av_L +f

        -- Backward pass
        df_do = crit:backward(outputs,target_table)
        rnn_lstm:backward(inputs_batch, df_do)
        
        grad_params:view(grad_params:size(1),1):renorm(1,2,5)
        
        rnn_lstm:updateParameters(lr)

    end
    
    new_L = av_L/math.floor(train:size(1)/b)
    
    print('Epoch '..i..': '..timer:time().real)
    print('Average Loss: '..new_L)
    
    if (new_L-old_L)< c then 
        lr = lr/2 
        c = c/2
    end
    
    if c < 0.001 then c = 0.1 end
    
end




Epoch 1: 75.165275812149	
Average Loss: 0.41858265751432	


Epoch 2: 73.915522098541	
Average Loss: 0.27539834276709	


Epoch 3: 73.694918870926	
Average Loss: 0.24231290195373	


Epoch 4: 73.67759180069	
Average Loss: 0.23135388427404	


Epoch 5: 73.235505819321	
Average Loss: 0.22704875186764	


...

Epoch 16: 73.073932170868	
Average Loss: 0.22341422233888	


Epoch 17: 72.55375289917	
Average Loss: 0.22341327133655	


Epoch 18: 76.872616052628	
Average Loss: 0.22341279579756	


Epoch 19: 73.501359939575	
Average Loss: 0.22341255802113	


Epoch 20: 73.484076023102	
Average Loss: 0.22341243913064	
